
# ETL – CMS HRRP Hospital Readmissions (Auto-Load FY CSVs)

This notebook auto-loads all CSVs in `../data/raw/`, infers the Fiscal Year (FY) from filenames, and produces Tableau-ready outputs for the two dashboards:
1) **National & State Trends**
2) **Hospital Distribution & Outliers**

Key behavior:
- Robust header normalization (case/underscores/squashed).
- Suppression-aware (flags & splits "Too Few to Report"/missing volume rows).
- Clean merges and saved aggregates for Tableau.


### Silent mode
Configure warnings/display to avoid noisy outputs or revealing local paths.

In [1]:

# Silence warnings (including chained assignment and pathy stack traces)
import warnings, pandas as _pd, os as _os
warnings.filterwarnings("ignore")
_pd.options.mode.chained_assignment = None
# Optional: cap display widths
_pd.set_option("display.max_colwidth", 120)

C:\Users\derek\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Step 1 — Imports & Configuration

In [2]:

from __future__ import annotations
import re, glob
from pathlib import Path
import pandas as pd
import numpy as np

RAW = Path("../data/raw")
PROC = Path("../data/processed")
PROC.mkdir(parents=True, exist_ok=True)

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)



## Step 2 — Auto-load all FY CSVs and tag metadata

- Loads every `*.csv` in `../data/raw/`
- Extracts FY from the filename (pattern like `fy_2024_...` or `FY_2025_...`)
- Stores temporary columns `__fy` and `__src`


In [3]:

def infer_fy_from_name(name: str) -> int | None:
    m = re.search(r"[Ff][Yy]_?(\d{4})", name)
    return int(m.group(1)) if m else None

paths = sorted(RAW.glob("*.csv"))
assert paths, f"No CSV files found in {RAW.resolve()}"

raw_frames = []
for p in paths:
    fy = infer_fy_from_name(p.name)
    df = pd.read_csv(p, dtype=str)
    df["__fy"] = fy
    df["__src"] = p.name
    raw_frames.append(df)

[(f["__src"].iloc[0], f["__fy"].iloc[0], f.shape) for f in raw_frames]


[('fy_2021_hospital_readmissions_reduction_program_hospital.csv',
  2021,
  (19224, 14)),
 ('fy_2022_hospital_readmissions_reduction_program_hospital.csv',
  2022,
  (19020, 14)),
 ('fy_2023_hospital_readmissions_reduction_program_hospital.csv',
  2023,
  (18990, 14)),
 ('fy_2024_hospital_readmissions_reduction_program_hospital.csv',
  2024,
  (18774, 14)),
 ('FY_2025_Hospital_Readmissions_Reduction_Program_Hospital.csv',
  2025,
  (18510, 14))]


## Step 3 — Normalize schema (by header name)

We canonicalize headers by:
- lowercasing
- stripping non-alphanumerics to create a key
- mapping to a canonical snake_case schema


In [4]:

def canon_key(s: str) -> str:
    return re.sub(r"[^a-z0-9]", "", s.lower())

MAP = {
    "facilityname": "facility_name",
    "facilityid": "facility_id",
    "state": "state",
    "measurename": "measure_name",
    "numberofdischarges": "number_of_discharges",
    "numberofreadmissions": "number_of_readmissions",
    "predictedreadmissionrate": "predicted_readmission_rate",
    "expectedreadmissionrate": "expected_readmission_rate",
    "excessreadmissionratio": "excess_readmission_ratio",
    "startdate": "start_date",
    "enddate": "end_date",
    "footnote": "footnote",
}

CANON_COLS = [
    "facility_id","facility_name","state","measure_name",
    "number_of_discharges","number_of_readmissions",
    "predicted_readmission_rate","expected_readmission_rate","excess_readmission_ratio",
    "start_date","end_date","footnote","__fy","__src"
]

norm_frames = []
for df in raw_frames:
    ren = {c: MAP.get(canon_key(c), c) for c in df.columns}
    df = df.rename(columns=ren).copy()
    # ensure presence
    for col in CANON_COLS:
        if col not in df.columns:
            df[col] = pd.NA
    df = df[CANON_COLS]
    norm_frames.append(df)

norm_frames[0].columns.tolist()


['facility_id',
 'facility_name',
 'state',
 'measure_name',
 'number_of_discharges',
 'number_of_readmissions',
 'predicted_readmission_rate',
 'expected_readmission_rate',
 'excess_readmission_ratio',
 'start_date',
 'end_date',
 'footnote',
 '__fy',
 '__src']


## Step 4 — Quick validation snapshots (coverage & missingness)


In [5]:

for df in norm_frames:
    fy = df["__fy"].iloc[0]
    print(f"=== FY {fy} ===")
    print("Rows:", len(df))
    print("States:", df["state"].nunique(), "Measures:", df["measure_name"].nunique())
    print(df[["number_of_discharges","number_of_readmissions",
              "predicted_readmission_rate","expected_readmission_rate","excess_readmission_ratio"]]
          .isna().mean().round(3))


=== FY 2021 ===
Rows: 19224
States: 51 Measures: 6
number_of_discharges          0.415
number_of_readmissions        0.422
predicted_readmission_rate    0.278
expected_readmission_rate     0.278
excess_readmission_ratio      0.278
dtype: float64
=== FY 2022 ===
Rows: 19020
States: 51 Measures: 6
number_of_discharges          0.460
number_of_readmissions        0.468
predicted_readmission_rate    0.297
expected_readmission_rate     0.297
excess_readmission_ratio      0.297
dtype: float64
=== FY 2023 ===
Rows: 18990
States: 51 Measures: 6
number_of_discharges          0.508
number_of_readmissions        0.519
predicted_readmission_rate    0.320
expected_readmission_rate     0.320
excess_readmission_ratio      0.320
dtype: float64
=== FY 2024 ===
Rows: 18774
States: 51 Measures: 6
number_of_discharges          0.569
number_of_readmissions        0.580
predicted_readmission_rate    0.357
expected_readmission_rate     0.357
excess_readmission_ratio      0.357
dtype: float64
=== FY 2025 ===



## Step 5 — Clean types, flag suppression, split kept vs suppressed

Rules:
- `suppressed=True` if (`number_of_discharges` **or** `number_of_readmissions` is null) **or** any cell in the row contains `"Too Few to Report"`.
- Parse dates, coerce numerics.
- Split into:
  - **kept**: rows suitable for analysis (metrics present)
  - **suppressed**: rows excluded from metrics (kept for transparency)


In [6]:

kept_frames = []
supp_frames = []

for df in norm_frames:
    # detect 'Too Few to Report' in any column
    too_few_any = df.astype(str).apply(lambda col: col.str.contains("Too Few", na=False)).any(axis=1)

    suppressed = (
        df["number_of_discharges"].isna() |
        df["number_of_readmissions"].isna() |
        too_few_any
    )
    df["suppressed"] = suppressed

    # parse dates
    df["start_date"] = pd.to_datetime(df["start_date"], errors="coerce", infer_datetime_format=True)
    df["end_date"]   = pd.to_datetime(df["end_date"],   errors="coerce", infer_datetime_format=True)

    # numerics
    for c in ["number_of_discharges","number_of_readmissions",
              "predicted_readmission_rate","expected_readmission_rate","excess_readmission_ratio"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    kept = df[~df["suppressed"]].copy()
    supp = df[df["suppressed"]].copy()

    # add FY
    kept["FY"] = df["__fy"].iloc[0]
    supp["FY"] = df["__fy"].iloc[0]

    # drop helpers
    for fr in (kept, supp):
        fr.drop(columns=["__fy","__src"], inplace=True, errors="ignore")

    # suppressed: drop metric cols
    METRICS = ["number_of_discharges","number_of_readmissions",
               "predicted_readmission_rate","expected_readmission_rate","excess_readmission_ratio"]
    supp = supp.drop(columns=METRICS + ["suppressed","footnote"], errors="ignore")

    # kept: drop flags & footnote; enforce metrics present
    kept = kept.drop(columns=["suppressed","footnote"], errors="ignore")
    kept = kept.dropna(subset=["excess_readmission_ratio"])

    kept_frames.append(kept)
    supp_frames.append(supp)

# sanity
kept_frames[0].head(2), supp_frames[0].head(2)


(  facility_id                     facility_name state            measure_name  number_of_discharges  \
 0       10001  SOUTHEAST ALABAMA MEDICAL CENTER    AL  READM-30-HIP-KNEE-HRRP                 258.0   
 1       10001  SOUTHEAST ALABAMA MEDICAL CENTER    AL      READM-30-CABG-HRRP                 268.0   
 
    number_of_readmissions  predicted_readmission_rate  expected_readmission_rate  excess_readmission_ratio start_date  \
 0                    17.0                      5.4150                     4.6146                    1.1735 2016-07-01   
 1                    41.0                     13.8076                    11.6339                    1.1868 2016-07-01   
 
     end_date    FY  
 0 2019-06-30  2021  
 1 2019-06-30  2021  ,
   facility_id             facility_name state        measure_name start_date   end_date    FY
 7       10005  MARSHALL MEDICAL CENTERS    AL   READM-30-AMI-HRRP 2016-07-01 2019-06-30  2021
 9       10005  MARSHALL MEDICAL CENTERS    AL  READM-30-CABG


## Step 6 — Combine & persist; suppression summary


In [7]:
kept_all = pd.concat(kept_frames, ignore_index=True)
supp_all = pd.concat(supp_frames, ignore_index=True)

# Summary by FY
summ = []
for df in norm_frames:
    fy = df["__fy"].iloc[0]
    total = len(df)
    suppressed_cnt = int(df["suppressed"].sum())
    summ.append({
        "FY": fy,
        "total_rows": total,
        "suppressed_rows": suppressed_cnt,
        "kept_rows": total - suppressed_cnt,
        "pct_suppressed": round(100 * suppressed_cnt / total, 2)
    })
suppression_summary = pd.DataFrame(summ).sort_values("FY")

# Save (CSV)
( PROC / "readmissions_cleaned.csv" ).write_text("")  # ensure path exists cross-platform (noop if exists)
kept_all.to_csv(PROC / "readmissions_cleaned.csv", index=False)
supp_all.to_csv(PROC / "readmissions_suppressed.csv", index=False)
suppression_summary.to_csv(PROC / "suppression_summary.csv", index=False)

print("Saved:", 
      PROC / "readmissions_cleaned.csv", 
      PROC / "readmissions_suppressed.csv", 
      PROC / "suppression_summary.csv")
kept_all.shape, supp_all.shape, suppression_summary

Saved: ..\data\processed\readmissions_cleaned.csv ..\data\processed\readmissions_suppressed.csv ..\data\processed\suppression_summary.csv


((46376, 12),
 (48142, 7),
      FY  total_rows  suppressed_rows  kept_rows  pct_suppressed
 0  2021       19224             8121      11103           42.24
 1  2022       19020             8900      10120           46.79
 2  2023       18990             9848       9142           51.86
 3  2024       18774            10884       7890           57.97
 4  2025       18510            10389       8121           56.13)


## Step 7 — Tableau-ready aggregates
- **state_summary.csv**: FY × state × measure with avg ERR, counts, and delta vs prior.
- **national_summary.csv**: FY × measure with national avg ERR.
- **hospital_summary.csv**: row-level for distributions/outliers.


In [11]:
# State summary
state_summary = (
    kept_all.groupby(["FY","state","measure_name"], dropna=False)
            .agg(
                avg_err=("excess_readmission_ratio","mean"),
                hospitals_reporting=("facility_id","nunique"),
                total_rows=("facility_id","size")
            )
            .reset_index()
            .sort_values(["FY","measure_name","state"])
)
state_summary["delta_prev"] = (
    state_summary.groupby(["state","measure_name"])["avg_err"].diff()
)

# National summary (mean of state averages)
national_summary = (
    state_summary.groupby(["FY","measure_name"], dropna=False)
                 .agg(national_avg_err=("avg_err","mean"))
                 .reset_index()
                 .sort_values(["FY","measure_name"])
)

# Hospital summary
hospital_summary = kept_all.loc[:, [
    "FY","facility_id","facility_name","state","measure_name",
    "excess_readmission_ratio","number_of_discharges","number_of_readmissions"
]].copy()
hospital_summary = hospital_summary.rename(columns={"excess_readmission_ratio": "err"})
hospital_summary["rank_by_err"] = (
    hospital_summary.groupby(["FY","measure_name"])["err"]
                    .rank(method="dense", ascending=True)
)

# Save (CSV)
state_summary.to_csv(PROC / "state_summary.csv", index=False)
national_summary.to_csv(PROC / "national_summary.csv", index=False)
hospital_summary.to_csv(PROC / "hospital_summary.csv", index=False)

print("Saved:", 
      PROC / "state_summary.csv", 
      PROC / "national_summary.csv", 
      PROC / "hospital_summary.csv")
state_summary.shape, national_summary.shape, hospital_summary.shape

Saved: ..\data\processed\state_summary.csv ..\data\processed\national_summary.csv ..\data\processed\hospital_summary.csv


((1496, 7), (30, 3), (46376, 9))